# SQL

Моя компания решила купить крупный сервис для чтения книг по подписке.\
`Задача аналитика`: проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг для сформулирования ценностное предложение для нового продукта.

**Оглавление:**

    1  Загрузим данные и подготовим их к анализу:
    1.1  Пропишем доступ к базе данных;
    1.2  Откроем файл с данными и изучем общую информацию.
    2  Задачи:
    2.1  Посчитаем, сколько книг вышло после 1 января 2000 года;
    2.2  Для каждой книги посчитаем количество обзоров и среднюю оценку;
    2.3  Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
    2.4 Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;
    2.5 Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
    3  Общие выводы.



## Загрузим данные и подготовим их к анализу:
### Пропишем доступ к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Откроем файл с данными и изучем общую информацию

In [3]:
# выведем первые строки таблицы books'
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` cодержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.\
По ключам `book_id`, `author_id`, `publisher_id` таблица связана с другими датафреймами. 

In [4]:
# выведем первые строки таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` cодержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.\
По ключу `author_id` таблица связана с датафреймом `books`. 

In [5]:
# выведем первые строки таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` cодержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.\
По ключу `publisher_id` таблица связана с датафреймом `books`. 

In [6]:
# выведем первые строки таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` cодержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.\
По ключу `book_id` таблица связана с датафреймами `books` и `reviews`. 

In [7]:
#  отображение всех столбцов полностью
pd.set_option('display.max_colwidth', 0)

# выведем первые строки таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


Таблица `reviews` cодержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.\
По ключу `book_id` таблица связана с датафреймами `books` и `ratings`. 

In [8]:
#создадим список с датасетами
df_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for name in df_list:
    print(f'\n Первые 5 строчек датасета {name}')
    query = ''f'SELECT * FROM {name} LIMIT 5'''
    con=engine.connect()
    display(pd.io.sql.read_sql(sql=text(query), con = con)) 


 Первые 5 строчек датасета books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



 Первые 5 строчек датасета authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



 Первые 5 строчек датасета publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



 Первые 5 строчек датасета ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



 Первые 5 строчек датасета reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


## Задачи:
### Посчитаем, сколько книг вышло после 1 января 2000 года


In [9]:
# выведем результат запроса
query = '''SELECT COUNT(book_id) 
           FROM books 
           WHERE CAST (publication_date AS date) > '2000-01-01'
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Большая часть книг из представленных для анализа вышло после 1 января 2000 года, а именно 819 книг из 1000 представленных.

### Для каждой книги посчитаем количество обзоров и среднюю оценку


In [10]:
# выведем результат запроса
query = '''SELECT b.title,
                count_review,
                rate
           FROM books b
           LEFT JOIN (SELECT book_id,
                          COUNT(review_id) AS count_review
                          FROM reviews 
                          GROUP BY book_id
                          ORDER BY count_review DESC) AS r ON b.book_id=r.book_id
           LEFT JOIN (SELECT book_id,
                          ROUND(AVG(rating), 2) AS rate
                      FROM ratings
                      GROUP BY book_id
                      ORDER BY book_id) AS ra ON b.book_id=ra.book_id
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,count_review,rate
0,'Salem's Lot,2.0,3.67
1,1 000 Places to See Before You Die,1.0,2.50
2,13 Little Blue Envelopes (Little Blue Envelope #1),3.0,4.67
3,1491: New Revelations of the Americas Before Columbus,2.0,4.50
4,1776,4.0,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.67
996,Xenocide (Ender's Saga #3),3.0,3.40
997,Year of Wonders,4.0,3.20
998,You Suck (A Love Story #2),2.0,4.50


Количество обзоров и среднюю оценку можно выводить рядом с названием книги на главном экране. Это информация поможет читателям при выборе книг.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [11]:
# выведем результат запроса
query = '''SELECT p.publisher,
                COUNT(b.book_id) AS count_book
           FROM books b
           JOIN publishers p ON b.publisher_id=p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY count_book DESC
           LIMIT 1
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_book
0,Penguin Books,42


Через наше приложение с издательством Penguin Books можно создать совместную рассылку уведомлений о новых выпущенных книгах.


### Определим автора с самой высокой средней оценкой книг — учитываеме только книги с 50 и более оценками

In [12]:
# выведем результат запроса
query = '''WITH temp AS (SELECT a.author,
                                b.book_id,
                             COUNT(r.rating_id),
                             AVG(r.rating) AS avg_rating
                         FROM ratings r
                         JOIN books b ON b.book_id=r.book_id
                         JOIN authors a ON b.author_id=a.author_id
                         GROUP BY a.author, b.book_id
                         HAVING COUNT(r.rating_id) >= 50
                         ORDER BY avg_rating DESC)
           SELECT author, 
               ROUND(AVG(avg_rating), 4) AS rate
           FROM temp
           GROUP BY author
           ORDER BY rate DESC
           LIMIT 1
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,rate
0,J.K. Rowling/Mary GrandPré,4.2838


В нашем сервисе необходимо разместить книги автора J.K. Rowling/Mary GrandPré, они пользуются большой популярностью.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [13]:
# выведем результат запроса
query = '''WITH rating_username AS (SELECT DISTINCT username,
                                        COUNT(rating_id)
                                    FROM ratings 
                                    GROUP BY username
                                    HAVING COUNT(rating_id) > 48),
           reviews_username AS (SELECT DISTINCT username,
                                    COUNT(review_id) AS count_review
                                    FROM reviews 
                                    GROUP BY username)
           SELECT AVG(reu.count_review)
           FROM reviews_username reu    
           JOIN rating_username rau ON reu.username=rau.username
         '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


В приложении можно создать мотивацию для пользователей, ставящих оценки и пишущих обзоры, такую как скидку на подписку при соблюдении ряда условий (например, имеющие количество обзоров больше 24).

## Общие выводы

**Мы проанализировали базу данных сервиса для чтения книг и на основании этого сделали выводы:**
- в базе представлены в основном современные книги, выпущенные после 1 января 2000 года - 819 штук;
- для каждой книги было посчитано количество обзоров и среднюяя оценка;
- издательство, которое выпустило наибольшее число книг толще 50 страниц, не считая брошюр - Penguin Books;
- автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré;
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.


**Рекомендации бизнесу:**
- разбавить сервис книгами классическими - выпущенными ранее 1 января 2000 года, например школьную классику;
- количество обзоров и среднюю оценку можно выводить рядом с названием книги на главном экране. Это информация поможет читателям при выборе книг;
- через наше приложение с издательством Penguin Books можно создать совместную рассылку уведомлений о новых выпущенных книгах;
- в нашем сервисе необходимо разместить книги автора J.K. Rowling/Mary GrandPré, они пользуются большой популярностью;
- в приложении можно создать мотивацию для пользователей, ставящих оценки и пишущих обзоры, такую как скидку на подписку при соблюдении ряда условий (например, имеющие количество обзоров больше 24).